In [119]:
import pyedflib
import pandas as pd
import numpy as np
import os
import scipy.signal
import neurokit2 as nk
from shapely.geometry import Polygon
import math
import re

In [120]:
def read_edf_file(file_path):
    f = pyedflib.EdfReader(file_path)
    file_header = f.getHeader()
    signal_headers = [f.getLabel(i) for i in range(f.signals_in_file)]
    signals = [f.readSignal(i) for i in range(f.signals_in_file)]
    f.close()
    return file_header, signal_headers, signals

In [121]:
# функции из func.py
#вычисление площади петли вектора сердечной активности
def calculate_area(points):
    polygon = Polygon(points)
    return polygon.area

#pассчитывает площади петель QRS или ST-T на проекциях
def loop(df_term, name, plotly_figures, show=False):
    points_frontal = list(zip(df_term['y'], df_term['z']))
    points_sagittal = list(zip(df_term['x'], df_term['z']))
    points_axial = list(zip(df_term['y'], df_term['x']))
    
    area_frontal = calculate_area(points_frontal)
    area_sagittal = calculate_area(points_sagittal)
    area_axial = calculate_area(points_axial)
    
    return area_frontal, area_sagittal, area_axial

#pассчитывает площади петель QRS или ST-T на проекциях
def get_area(show, df, waves_peak, start, Fs_new, QRS, T, plotly_figures):
    area = []
    waves_peak['ECG_Q_Peaks'] = [x for x in waves_peak['ECG_Q_Peaks'] if not math.isnan(x)]
    waves_peak['ECG_S_Peaks'] = [x for x in waves_peak['ECG_S_Peaks'] if not math.isnan(x)]
    waves_peak['ECG_T_Offsets'] = [x for x in waves_peak['ECG_T_Offsets'] if not math.isnan(x)]

    closest_Q_peak = min(waves_peak['ECG_Q_Peaks'], key=lambda x: abs(x - start))
    closest_S_peak = min(waves_peak['ECG_S_Peaks'], key=lambda x: abs(x - start))
    df_term_qrs = df.iloc[closest_Q_peak:closest_S_peak, :]
    
    if QRS:
        area_qrs = list(loop(df_term_qrs, name='QRS', plotly_figures=plotly_figures, show=show))
        area.extend(area_qrs)

    closest_T_end = min(waves_peak['ECG_T_Offsets'], key=lambda x: abs(x - closest_S_peak))
    df_term_st = df.iloc[closest_S_peak + int(0.025*Fs_new):closest_T_end, :]
    
    if T:
        area_st = list(loop(df_term_st, name='T', plotly_figures=plotly_figures, show=show))
        area.extend(area_st)
    
    return area
#угол между векторами средней электродвижущей силы QRS и ST-T сегментов.
def find_qrst_angle(mean_qrs, mean_t, name=''):
    mean_qrs = np.array(mean_qrs)
    mean_t = np.array(mean_t)
    dot_product = np.dot(mean_qrs, mean_t)
    norm_qrs = np.linalg.norm(mean_qrs)
    norm_t = np.linalg.norm(mean_t)
    angle_radians = np.arccos(dot_product / (norm_qrs * norm_t))
    return np.degrees(angle_radians)


def extract_patient_info(file_header):
    """
    Извлекает пол, возраст и код пациента из заголовка EDF файла.
    """
    gender = file_header.get('sex') or file_header.get('gender', 'Unknown')

    male = 1 if 'M' in str(gender) else 0
    female = 1 if 'F' in str(gender) else 0
    
    start_date = file_header.get('startdate', None)
    age = 'Unknown'
    birthdate = file_header.get('birthdate', None)
    
    if birthdate and start_date:
        birth_year = int(birthdate[-4:])
        age = start_date.year - birth_year
    
    admin_code = file_header.get('admincode', 'Unknown')

    return male, female, age, admin_code




In [122]:
#преобразование стандартных ЭКГ сигналов в ВЭКГ, добавление координат X, Y, Z
def make_vecg(df_term):
    """
    Преобразует сигналы ЭКГ в ВЭКГ и добавляет координаты x, y, z.
    """
    DI = df_term['ECG I']
    DII = df_term['ECG II']
    V1 = df_term['ECG V1']
    V2 = df_term['ECG V2']
    V3 = df_term['ECG V3']
    V4 = df_term['ECG V4']
    V5 = df_term['ECG V5']
    V6 = df_term['ECG V6']

#координаты x, y, z для ВЭКГ
    df_term['x'] = -(-0.172*V1 - 0.074*V2 + 0.122*V3 + 0.231*V4 + 0.239*V5 + 0.194*V6 + 0.156*DI - 0.01*DII)
    df_term['y'] = (0.057*V1 - 0.019*V2 - 0.106*V3 - 0.022*V4 + 0.041*V5 + 0.048*V6 - 0.227*DI + 0.887*DII)
    df_term['z'] = -(-0.229*V1 - 0.31*V2 - 0.246*V3 - 0.063*V4 + 0.055*V5 + 0.108*V6 + 0.022*DI + 0.102*DII)
    
    return df_term

In [123]:
# дополнительные признаки
#эксцентриситет петли на основе ковариационной матрицы для анализа формы петли

def eccentricity_loop(x, y):
    cov_matrix = np.cov(x, y)
    eigenvalues, _ = np.linalg.eig(cov_matrix)
    return np.sqrt(1 - (min(eigenvalues) / max(eigenvalues)))

# средняя и максимальная скорость вектора ЭДС
def calculate_velocity(df_term):
    dx = np.diff(df_term['x'])
    dy = np.diff(df_term['y'])
    dz = np.diff(df_term['z'])
    dt = np.diff(df_term['time'])
    velocity = np.sqrt(dx**2 + dy**2 + dz**2) / dt
    return np.mean(velocity), np.max(velocity)

#продолжительность волн QRS и RR.
def calculate_wave_durations(waves_peak, Fs_new):
    durations = {}
    if 'ECG_R_Peaks' in waves_peak:
        r_peaks = waves_peak['ECG_R_Peaks']
        rr_intervals = np.diff(r_peaks) / Fs_new
        durations['RR_interval_median'] = np.median(rr_intervals)
    
    if 'ECG_T_Offsets' in waves_peak and 'ECG_Q_Peaks' in waves_peak:
        qrs_duration = (waves_peak['ECG_T_Offsets'] - waves_peak['ECG_Q_Peaks']) / Fs_new
        durations['QRS_duration'] = np.median(qrs_duration)
    
    return durations



In [124]:

def process_edf_file(file_path):
    
    file_header, signal_headers, signals = read_edf_file(file_path)
    
    male, female, age, admin_code = extract_patient_info(file_header)
  
    df = pd.DataFrame(np.array(signals).T, columns=signal_headers)
    df['time'] = np.arange(len(df)) * (1 / 700)

    # Преобразование ЭКГ в ВЭКГ
    df = make_vecg(df)

    #очистка сигнала и поиск пиков
    signal = nk.ecg_clean(df['ECG I'], sampling_rate=500)
    _, rpeaks = nk.ecg_peaks(signal, sampling_rate=500)
    _, waves_peak = nk.ecg_delineate(signal, rpeaks, sampling_rate=500, method="peak")

    #площади и углы
    start = rpeaks['ECG_R_Peaks'][0]
    areas = get_area(show=False, df=df, waves_peak=waves_peak, start=start, Fs_new=500, QRS=True, T=True, plotly_figures=[])

    Square_QRS_frontal, Square_QRS_sagittal, Square_QRS_axial = areas[:3]
    Square_ST_frontal, Square_ST_sagittal, Square_ST_axial = areas[3:]
    
    mean_qrs = [df['x'].mean(), df['y'].mean(), df['z'].mean()]
    mean_t = [df['x'].mean(), df['y'].mean(), df['z'].mean()]
    Frontal_Angle_QRST = find_qrst_angle(mean_qrs[1:], mean_t[1:])

    # дополнительные признаки
    eccentricity_qrs = eccentricity_loop(df['x'], df['y'])
    mean_velocity, max_velocity = calculate_velocity(df)
    durations = calculate_wave_durations(waves_peak, 500)

    features = {
        'File_Name': os.path.basename(file_path),
        'Male': male,  # Добавляем признак Male
        'Female': female,      
        'Age': age,             
        'Class': admin_code, 
        'Square_QRS_frontal': Square_QRS_frontal,
        'Square_QRS_sagittal': Square_QRS_sagittal,
        'Square_QRS_axial': Square_QRS_axial,
        'Square_ST_frontal': Square_ST_frontal,
        'Square_ST_sagittal': Square_ST_sagittal,
        'Square_ST_axial': Square_ST_axial,
        'Frontal_Angle_QRST': Frontal_Angle_QRST,
        'Eccentricity_QRS': eccentricity_qrs,
        'Mean_Velocity': mean_velocity,
        'Max_Velocity': max_velocity,
    }
    
    features.update(durations)
    
    return features


In [125]:
def process_folder(folder_path):
    all_features = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.edf'):
            file_path = os.path.join(folder_path, file_name)
            print(f"Обрабатываем файл: {file_name}")
            
            features = process_edf_file(file_path)
            all_features.append(features)

    features_df = pd.DataFrame(all_features)
    
    return features_df

In [126]:
def process_files_in_folder(folder_path, class_label):
    """
    Обрабатывает все EDF файлы в папке и добавляет им соответствующий класс.
    
    Args:
        folder_path (str): Путь к папке с EDF файлами.
        class_label (int): Метка класса (1 - амилоидоз, 2 - нет амилоидоза, 3 - подозрение).
        
    Returns:
        pd.DataFrame: DataFrame с признаками и меткой класса для всех файлов из папки.
    """
    all_features = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.edf'):
            file_path = os.path.join(folder_path, file_name)
            print(f"Обрабатываем файл: {file_name}")

            features = process_edf_file(file_path)

            features['Class'] = class_label
            
            all_features.append(features)

    return pd.DataFrame(all_features)


In [127]:
#def process_all_folders(amyloidosis_folder, no_amyloidosis_folder, suspect_folder):
def process_all_folders(amyloidosis_folder, no_amyloidosis_folder):
    """
    Обрабатывает три папки с файлами EDF, каждая из которых соответствует определенному классу.
    
    Args:
        amyloidosis_folder (str): Путь к папке с подтвержденным амилоидозом (класс 1).
        no_amyloidosis_folder (str): Путь к папке с файлами без амилоидоза (класс 2).
        suspect_folder (str): Путь к папке с подозрением на амилоидоз (класс 3).
        
    Returns:
        pd.DataFrame: DataFrame со всеми признаками для всех файлов из всех папок.
    """
    # файлы для каждого класса
    amyloidosis_df = process_files_in_folder(amyloidosis_folder, class_label=1)
    no_amyloidosis_df = process_files_in_folder(no_amyloidosis_folder, class_label=2)
    #suspect_df = process_files_in_folder(suspect_folder, class_label=3)

    # все результаты в один DataFrame
    all_data_df = pd.concat([amyloidosis_df, no_amyloidosis_df], ignore_index=True)
    #all_data_df = pd.concat([amyloidosis_df, no_amyloidosis_df, suspect_df], ignore_index=True)
    return all_data_df

In [128]:
def save_features_to_csv(features_df, output_path='features.csv'):
    """
    Сохраняет DataFrame с признаками в CSV файл.
    
    Args:
        features_df (pd.DataFrame): DataFrame с рассчитанными признаками.
        output_path (str): Путь для сохранения CSV файла.
    """
    features_df.to_csv(output_path, index=False)
    print(f"Признаки успешно сохранены в файл: {output_path}")

In [129]:
def calculate_wave_durations(waves_peak, Fs_new):
    """
    Рассчитывает интервалы волн, такие как продолжительность QRS и RR-интервалы.
    """
    durations = {}

    if 'ECG_R_Peaks' in waves_peak:
        r_peaks = waves_peak['ECG_R_Peaks']
        rr_intervals = np.diff(r_peaks) / Fs_new
        durations['RR_interval_median'] = np.median(rr_intervals)

    if 'ECG_T_Offsets' in waves_peak and 'ECG_Q_Peaks' in waves_peak:
        #убедимся, что количество пиков Q и T совпадает
        q_peaks = waves_peak['ECG_Q_Peaks']
        t_offsets = waves_peak['ECG_T_Offsets']
        if len(q_peaks) == len(t_offsets):
            qrs_duration = [(t - q) / Fs_new for t, q in zip(t_offsets, q_peaks)]
            durations['QRS_duration'] = np.median(qrs_duration)
        else:
            print("Количество Q пиков и T оффсетов не совпадает!")
    
    return durations


In [130]:
if __name__ == "__main__":

    amyloidosis_folder = 'D:/VECG/Data/DataAmy'
    no_amyloidosis_folder = 'D:/VECG/Data/DataNoAmy'
    #suspect_folder = 'D:/VECG/Data/DataMBAmy'
    
    output_file = 'D:/VECG/Features/features.csv'

    features_df = process_all_folders(amyloidosis_folder, no_amyloidosis_folder)
    #features_df = process_all_folders(amyloidosis_folder, no_amyloidosis_folder, suspect_folder)
    print("Таблица признаков:")
    print(features_df)
    features_df.to_csv(output_file, index=False)


Обрабатываем файл: Amy1.edf
Обрабатываем файл: Amy10.edf
Обрабатываем файл: Amy11.edf
Количество Q пиков и T оффсетов не совпадает!
Обрабатываем файл: Amy12.edf
Количество Q пиков и T оффсетов не совпадает!
Обрабатываем файл: Amy13.edf


C:\Users\Зина\AppData\Local\Temp\ipykernel_3000\1089762726.py:46: RuntimeWarning: invalid value encountered in arccos
  angle_radians = np.arccos(dot_product / (norm_qrs * norm_t))


Обрабатываем файл: Amy14.edf


C:\Users\Зина\AppData\Local\Temp\ipykernel_3000\1089762726.py:46: RuntimeWarning: invalid value encountered in arccos
  angle_radians = np.arccos(dot_product / (norm_qrs * norm_t))
C:\Users\Зина\AppData\Local\Temp\ipykernel_3000\1089762726.py:46: RuntimeWarning: invalid value encountered in arccos
  angle_radians = np.arccos(dot_product / (norm_qrs * norm_t))


Обрабатываем файл: Amy15.edf
Обрабатываем файл: Amy2.edf


...

Обрабатываем файл: AMYC2.edf
Обрабатываем файл: AMYC3.edf
Обрабатываем файл: AMYC4.edf


C:\Users\Зина\AppData\Local\Temp\ipykernel_3000\1089762726.py:46: RuntimeWarning: invalid value encountered in arccos
  angle_radians = np.arccos(dot_product / (norm_qrs * norm_t))


Обрабатываем файл: AMYC5.edf


C:\Users\Зина\AppData\Local\Temp\ipykernel_3000\1089762726.py:46: RuntimeWarning: invalid value encountered in arccos
  angle_radians = np.arccos(dot_product / (norm_qrs * norm_t))


Количество Q пиков и T оффсетов не совпадает!
Обрабатываем файл: AMYC6.edf
Обрабатываем файл: AMYC7.edf
Обрабатываем файл: AMYC8.edf
Обрабатываем файл: AMYC9.edf
Таблица признаков:
     File_Name  Male  Female  Age  Class  Square_QRS_frontal  \
0     Amy1.edf     1       0   68      1            0.026755   
1    Amy10.edf     0       1   52      1            0.003776   
2    Amy11.edf     1       0   64      1            0.020615   
3    Amy12.edf     0       1   76      1            0.456600   
4    Amy13.edf     0       1   80      1            0.145836   
5    Amy14.edf     1       0   65      1            0.009407   
6    Amy15.edf     1       0   67      1            0.223673   
7     Amy2.edf     1       0   64      1            0.036696   
8     Amy3.edf     1       0   56      1            0.522017   
9     Amy4.edf     0       1   70      1            0.009927   
10    Amy5.edf     1       0   83      1            0.609483   
11    Amy6.edf     0       1   61      1           

In [131]:
features_df.head()


,File_Name,Male,Female,Age,Class,Square_QRS_frontal,Square_QRS_sagittal,Square_QRS_axial,Square_ST_frontal,Square_ST_sagittal,Square_ST_axial,Frontal_Angle_QRST,Eccentricity_QRS,Mean_Velocity,Max_Velocity,QRS_duration
0,Amy1.edf,1,0,68,1,0.026755,0.240019,0.030144,0.005433,0.008091,0.008706,0.000000,0.959390,11.977465,165.604484,0.442
1,Amy10.edf,0,1,52,1,0.003776,0.079165,0.001862,0.000323,0.001717,0.002690,0.000001,0.954373,6.190198,112.144974,0.384
2,Amy11.edf,1,0,64,1,0.020615,0.398480,0.301191,0.004932,0.005629,0.003688,0.000000,0.944062,8.663327,157.271671,NaN
3,Amy12.edf,0,1,76,1,0.456600,0.040978,0.000796,0.009531,0.009618,0.000179,NaN,0.834735,8.063255,108.658630,NaN
4,Amy13.edf,0,1,80,1,0.145836,0.167594,0.112336,0.001844,0.004964,0.002840,0.000000,0.935549,13.396447,104.589631,0.396
